# Лабораторная работа 4

Tensorflow 2.x

1) Подготовка данных

2) Использование Keras Model API

3) Использование Keras Sequential + Functional API

https://www.tensorflow.org/tutorials

Для выполнения лабораторной работы необходимо установить tensorflow версии 2.0 или выше .

Рекомендуется использовать возможности Colab'а по обучению моделей на GPU.



In [2]:
import os
import tensorflow as tf
import numpy as np
import math
import timeit
import matplotlib.pyplot as plt

%matplotlib inline

2024-05-04 22:20:22.598221: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-04 22:20:22.683077: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-04 22:20:23.067152: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-04 22:20:24.439873: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# Подготовка данных
Загрузите набор данных из предыдущей лабораторной работы. 

In [3]:
def load_cifar10(num_training=49000, num_validation=1000, num_test=10000):
    """
    Fetch the CIFAR-10 dataset from the web and perform preprocessing to prepare
    it for the two-layer neural net classifier. These are the same steps as
    we used for the SVM, but condensed to a single function.
    """
    # Load the raw CIFAR-10 dataset and use appropriate data types and shapes
    cifar10 = tf.keras.datasets.cifar10.load_data()
    (X_train, y_train), (X_test, y_test) = cifar10
    X_train = np.asarray(X_train, dtype=np.float32)
    y_train = np.asarray(y_train, dtype=np.int32).flatten()
    X_test = np.asarray(X_test, dtype=np.float32)
    y_test = np.asarray(y_test, dtype=np.int32).flatten()

    # Subsample the data
    mask = range(num_training, num_training + num_validation)
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = range(num_training)
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = range(num_test)
    X_test = X_test[mask]
    y_test = y_test[mask]

    # Normalize the data: subtract the mean pixel and divide by std
    mean_pixel = X_train.mean(axis=(0, 1, 2), keepdims=True)
    std_pixel = X_train.std(axis=(0, 1, 2), keepdims=True)
    X_train = (X_train - mean_pixel) / std_pixel
    X_val = (X_val - mean_pixel) / std_pixel
    X_test = (X_test - mean_pixel) / std_pixel

    return X_train, y_train, X_val, y_val, X_test, y_test

# If there are errors with SSL downloading involving self-signed certificates,
# it may be that your Python version was recently installed on the current machine.
# See: https://github.com/tensorflow/tensorflow/issues/10779
# To fix, run the command: /Applications/Python\ 3.7/Install\ Certificates.command
#   ...replacing paths as necessary.

# Invoke the above function to get our data.
NHW = (0, 1, 2)
X_train, y_train, X_val, y_val, X_test, y_test = load_cifar10()
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape, y_train.dtype)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 41s 0us/step
Train data shape:  (49000, 32, 32, 3)
Train labels shape:  (49000,) int32
Validation data shape:  (1000, 32, 32, 3)
Validation labels shape:  (1000,)
Test data shape:  (10000, 32, 32, 3)
Test labels shape:  (10000,)


In [4]:
class Dataset(object):
    def __init__(self, X, y, batch_size, shuffle=False):
        """
        Construct a Dataset object to iterate over data X and labels y
        
        Inputs:
        - X: Numpy array of data, of any shape
        - y: Numpy array of labels, of any shape but with y.shape[0] == X.shape[0]
        - batch_size: Integer giving number of elements per minibatch
        - shuffle: (optional) Boolean, whether to shuffle the data on each epoch
        """
        assert X.shape[0] == y.shape[0], 'Got different numbers of data and labels'
        self.X, self.y = X, y
        self.batch_size, self.shuffle = batch_size, shuffle

    def __iter__(self):
        N, B = self.X.shape[0], self.batch_size
        idxs = np.arange(N)
        if self.shuffle:
            np.random.shuffle(idxs)
        return iter((self.X[i:i+B], self.y[i:i+B]) for i in range(0, N, B))


train_dset = Dataset(X_train, y_train, batch_size=64, shuffle=True)
val_dset = Dataset(X_val, y_val, batch_size=64, shuffle=False)
test_dset = Dataset(X_test, y_test, batch_size=64)

In [5]:
# We can iterate through a dataset like this:
for t, (x, y) in enumerate(train_dset):
    print(t, x.shape, y.shape)
    if t > 5: break

0 (64, 32, 32, 3) (64,)
1 (64, 32, 32, 3) (64,)
2 (64, 32, 32, 3) (64,)
3 (64, 32, 32, 3) (64,)
4 (64, 32, 32, 3) (64,)
5 (64, 32, 32, 3) (64,)
6 (64, 32, 32, 3) (64,)


#  Keras Model Subclassing API


Для реализации собственной модели с помощью Keras Model Subclassing API необходимо выполнить следующие шаги:

1) Определить новый класс, который является наследником tf.keras.Model.

2) В методе __init__() определить все необходимые слои из модуля tf.keras.layer

3) Реализовать прямой проход в методе call() на основе слоев, объявленных в __init__()

Ниже приведен пример использования keras API для определения двухслойной полносвязной сети. 

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras

In [7]:
device = '/cpu:0'

In [8]:
class TwoLayerFC(tf.keras.Model):
    def __init__(self, hidden_size, num_classes):
        super(TwoLayerFC, self).__init__()        
        initializer = tf.initializers.VarianceScaling(scale=2.0)
        self.fc1 = tf.keras.layers.Dense(hidden_size, activation='relu',
                                   kernel_initializer=initializer)
        self.fc2 = tf.keras.layers.Dense(num_classes, activation='softmax',
                                   kernel_initializer=initializer)
        self.flatten = tf.keras.layers.Flatten()
    
    def call(self, x, training=False):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)
        return x


def test_TwoLayerFC():
    """ A small unit test to exercise the TwoLayerFC model above. """
    input_size, hidden_size, num_classes = 50, 42, 10
    x = tf.zeros((64, input_size))
    model = TwoLayerFC(hidden_size, num_classes)
    with tf.device(device):
        scores = model(x)
        print(scores.shape)
        
test_TwoLayerFC()

(64, 10)


Реализуйте трехслойную CNN для вашей задачи классификации. 

Архитектура сети:
    
1. Сверточный слой (5 x 5 kernels, zero-padding = 'same')
2. Функция активации ReLU 
3. Сверточный слой (3 x 3 kernels, zero-padding = 'same')
4. Функция активации ReLU 
5. Полносвязный слой 
6. Функция активации Softmax 

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/Conv2D

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/Dense

In [9]:
class ThreeLayerConvNet(tf.keras.Model):
    def __init__(self, channel_1, channel_2, num_classes):
        super(ThreeLayerConvNet, self).__init__()
        ########################################################################
        # TODO: Implement the __init__ method for a three-layer ConvNet. You   #
        # should instantiate layer objects to be used in the forward pass.     #
        ########################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        self.conv1 = tf.keras.layers.Conv2D(channel_1, (5, 5), padding='same', activation='relu')
        self.conv2 = tf.keras.layers.Conv2D(channel_2, (3, 3), padding='same', activation='relu')
        self.flatten = tf.keras.layers.Flatten()
        self.fc = tf.keras.layers.Dense(num_classes, activation='softmax')

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ########################################################################
        #                           END OF YOUR CODE                           #
        ########################################################################
        
    def call(self, x, training=False):
        scores = None
        ########################################################################
        # TODO: Implement the forward pass for a three-layer ConvNet. You      #
        # should use the layer objects defined in the __init__ method.         #
        ########################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        scores = self.conv1(x)
        scores = self.conv2(scores)
        scores = self.flatten(scores)
        scores = self.fc(scores)

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ########################################################################
        #                           END OF YOUR CODE                           #
        ########################################################################        
        return scores

In [10]:
def test_ThreeLayerConvNet():    
    channel_1, channel_2, num_classes = 12, 8, 10
    model = ThreeLayerConvNet(channel_1, channel_2, num_classes)
    with tf.device(device):
        x = tf.zeros((64, 3, 32, 32))
        scores = model(x)
        print(scores.shape)

test_ThreeLayerConvNet()

(64, 10)


Пример реализации процесса обучения:

In [23]:
print_every = 50

In [24]:
def train_part34(model_init_fn, optimizer_init_fn, num_epochs=1, is_training=False):
    """
    Simple training loop for use with models defined using tf.keras. It trains
    a model for one epoch on the CIFAR-10 training set and periodically checks
    accuracy on the CIFAR-10 validation set.
    
    Inputs:
    - model_init_fn: A function that takes no parameters; when called it
      constructs the model we want to train: model = model_init_fn()
    - optimizer_init_fn: A function which takes no parameters; when called it
      constructs the Optimizer object we will use to optimize the model:
      optimizer = optimizer_init_fn()
    - num_epochs: The number of epochs to train for
    
    Returns: Nothing, but prints progress during trainingn
    """    
    with tf.device(device):

        
        loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
        
        model = model_init_fn()
        optimizer = optimizer_init_fn()
        
        train_loss = tf.keras.metrics.Mean(name='train_loss')
        train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
    
        val_loss = tf.keras.metrics.Mean(name='val_loss')
        val_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='val_accuracy')
        
        t = 0
        for epoch in range(num_epochs):
            
            # Reset the metrics - https://www.tensorflow.org/alpha/guide/migration_guide#new-style_metrics
            train_loss.reset_state()
            train_accuracy.reset_state()
            
            for x_np, y_np in train_dset:
                with tf.GradientTape() as tape:
                    
                    # Use the model function to build the forward pass.
                    scores = model(x_np, training=is_training)
                    loss = loss_fn(y_np, scores)
      
                    gradients = tape.gradient(loss, model.trainable_variables)
                    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
                    
                    # Update the metrics
                    train_loss.update_state(loss)
                    train_accuracy.update_state(y_np, scores)
                    
                    if t % print_every == 0:
                        val_loss.reset_state()
                        val_accuracy.reset_state()
                        for test_x, test_y in val_dset:
                            # During validation at end of epoch, training set to False
                            prediction = model(test_x, training=False)
                            t_loss = loss_fn(test_y, prediction)

                            val_loss.update_state(t_loss)
                            val_accuracy.update_state(test_y, prediction)
                        
                        template = 'Iteration {}, Epoch {}, Loss: {}, Accuracy: {}, Val Loss: {}, Val Accuracy: {}'
                        print (template.format(t, epoch+1,
                                             train_loss.result(),
                                             train_accuracy.result()*100,
                                             val_loss.result(),
                                             val_accuracy.result()*100))
                    t += 1

In [25]:
hidden_size, num_classes = 4000, 10
learning_rate = 1e-2

def model_init_fn():
    return TwoLayerFC(hidden_size, num_classes)

def optimizer_init_fn():
    return tf.keras.optimizers.SGD(learning_rate=learning_rate)

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 3.9485299587249756, Accuracy: 6.25, Val Loss: 2.9511518478393555, Val Accuracy: 12.700000762939453
Iteration 50, Epoch 1, Loss: 2.4450135231018066, Accuracy: 24.356616973876953, Val Loss: 2.246093988418579, Val Accuracy: 31.299999237060547
Iteration 100, Epoch 1, Loss: 2.2738003730773926, Accuracy: 28.511756896972656, Val Loss: 1.9143033027648926, Val Accuracy: 37.5
Iteration 150, Epoch 1, Loss: 2.1631040573120117, Accuracy: 30.691225051879883, Val Loss: 1.8723206520080566, Val Accuracy: 39.39999771118164
Iteration 200, Epoch 1, Loss: 2.093363046646118, Accuracy: 32.4238166809082, Val Loss: 1.842711329460144, Val Accuracy: 40.20000076293945
Iteration 250, Epoch 1, Loss: 2.0527267456054688, Accuracy: 33.291831970214844, Val Loss: 1.8414257764816284, Val Accuracy: 38.29999923706055
Iteration 300, Epoch 1, Loss: 2.013272762298584, Accuracy: 34.156978607177734, Val Loss: 1.9414770603179932, Val Accuracy: 36.099998474121094
Iteration 350, Epoch 1, Loss: 1.9795148

Обучите трехслойную CNN. В tf.keras.optimizers.SGD укажите Nesterov momentum = 0.9 . 

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/optimizers/SGD

Значение accuracy на валидационной выборке после 1 эпохи обучения должно быть > 50% .

In [26]:
learning_rate = 3e-3
channel_1, channel_2, num_classes = 32, 16, 10

def model_init_fn():
    model = None
    ############################################################################
    # TODO: Complete the implementation of model_fn.                           #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    model = ThreeLayerConvNet(channel_1, channel_2, num_classes)

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                           END OF YOUR CODE                               #
    ############################################################################
    return model

def optimizer_init_fn():
    optimizer = None
    ############################################################################
    # TODO: Complete the implementation of model_fn.                           #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.9, nesterov=True)

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                           END OF YOUR CODE                               #
    ############################################################################
    return optimizer

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 2.287132740020752, Accuracy: 7.8125, Val Loss: 2.294034719467163, Val Accuracy: 13.600000381469727
Iteration 50, Epoch 1, Loss: 2.023817777633667, Accuracy: 27.971813201904297, Val Loss: 1.8539752960205078, Val Accuracy: 34.099998474121094
Iteration 100, Epoch 1, Loss: 1.881248116493225, Accuracy: 32.82796859741211, Val Loss: 1.6602815389633179, Val Accuracy: 42.099998474121094
Iteration 150, Epoch 1, Loss: 1.799367904663086, Accuracy: 36.092716217041016, Val Loss: 1.6150872707366943, Val Accuracy: 44.0
Iteration 200, Epoch 1, Loss: 1.743436574935913, Accuracy: 38.24626922607422, Val Loss: 1.4907405376434326, Val Accuracy: 49.20000076293945
Iteration 250, Epoch 1, Loss: 1.7042635679244995, Accuracy: 39.70991134643555, Val Loss: 1.4687167406082153, Val Accuracy: 49.39999771118164
Iteration 300, Epoch 1, Loss: 1.6621670722961426, Accuracy: 41.21677780151367, Val Loss: 1.464042067527771, Val Accuracy: 48.69999694824219
Iteration 350, Epoch 1, Loss: 1.6295042037

# Использование Keras Sequential API для реализации последовательных моделей.

Пример для полносвязной сети:

In [29]:
learning_rate = 1e-2

def model_init_fn():
    input_shape = (32, 32, 3)
    hidden_layer_size, num_classes = 4000, 10
    initializer = tf.initializers.VarianceScaling(scale=2.0)
    layers = [
        tf.keras.layers.Flatten(input_shape=input_shape),
        tf.keras.layers.Dense(hidden_layer_size, activation='relu',
                              kernel_initializer=initializer),
        tf.keras.layers.Dense(num_classes, activation='softmax', 
                              kernel_initializer=initializer),
    ]
    model = tf.keras.Sequential(layers)
    return model

def optimizer_init_fn():
    return tf.keras.optimizers.SGD(learning_rate=learning_rate) 

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 3.149225950241089, Accuracy: 4.6875, Val Loss: 2.88926100730896, Val Accuracy: 14.0
Iteration 50, Epoch 1, Loss: 2.364603042602539, Accuracy: 25.42892074584961, Val Loss: 2.219413995742798, Val Accuracy: 32.10000228881836
Iteration 100, Epoch 1, Loss: 2.2400622367858887, Accuracy: 28.341585159301758, Val Loss: 1.9270519018173218, Val Accuracy: 37.099998474121094
Iteration 150, Epoch 1, Loss: 2.142667770385742, Accuracy: 30.556705474853516, Val Loss: 1.9186222553253174, Val Accuracy: 37.400001525878906
Iteration 200, Epoch 1, Loss: 2.081268548965454, Accuracy: 31.988494873046875, Val Loss: 1.8323907852172852, Val Accuracy: 39.5
Iteration 250, Epoch 1, Loss: 2.0408129692077637, Accuracy: 32.9681282043457, Val Loss: 1.8321492671966553, Val Accuracy: 39.20000076293945
Iteration 300, Epoch 1, Loss: 2.0040981769561768, Accuracy: 33.81956100463867, Val Loss: 1.8861968517303467, Val Accuracy: 39.5
Iteration 350, Epoch 1, Loss: 1.9709477424621582, Accuracy: 34.731124

Альтернативный менее гибкий способ обучения:

In [30]:
model = model_init_fn()
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
              loss='sparse_categorical_crossentropy',
              metrics=[tf.keras.metrics.sparse_categorical_accuracy])
model.fit(X_train, y_train, batch_size=64, epochs=1, validation_data=(X_val, y_val))
model.evaluate(X_test, y_test)

2024-05-04 22:50:21.350413: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 602112000 exceeds 10% of free system memory.


766/766 ━━━━━━━━━━━━━━━━━━━━ 60s 77ms/step - loss: 2.0207 - sparse_categorical_accuracy: 0.3387 - val_loss: 1.8205 - val_sparse_categorical_accuracy: 0.4170
313/313 ━━━━━━━━━━━━━━━━━━━━ 6s 20ms/step - loss: 1.7689 - sparse_categorical_accuracy: 0.4153


[1.7645881175994873, 0.4169999957084656]

Перепишите реализацию трехслойной CNN с помощью tf.keras.Sequential API . Обучите модель двумя способами.

In [32]:
def model_init_fn():
    model = None
    ############################################################################
    # TODO: Construct a three-layer ConvNet using tf.keras.Sequential.         #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    model = tf.keras.Sequential([
        # Первый сверточный слой
        tf.keras.layers.Conv2D(32, (5, 5), padding='same', activation='relu', input_shape=(32, 32, 3)),
        # Первый пулинг слой
        tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2)),
        # Второй сверточный слой
        tf.keras.layers.Conv2D(64, (5, 5), padding='same', activation='relu'),
        # Второй пулинг слой
        tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2)),
        # Разворачиваем данные перед подачей на полносвязный слой
        tf.keras.layers.Flatten(),
        # Полносвязный слой с relu активацией
        tf.keras.layers.Dense(1024, activation='relu'),
        # Выходной слой с softmax активацией
        tf.keras.layers.Dense(10, activation='softmax')
    ])

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                            END OF YOUR CODE                              #
    ############################################################################
    return model

learning_rate = 5e-4
def optimizer_init_fn():
    optimizer = None
    ############################################################################
    # TODO: Complete the implementation of model_fn.                           #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    optimizer = tf.keras.optimizers.SGD(learning_rate)

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                           END OF YOUR CODE                               #
    ############################################################################
    return optimizer

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 2.3131260871887207, Accuracy: 14.0625, Val Loss: 2.3085272312164307, Val Accuracy: 12.700000762939453
Iteration 50, Epoch 1, Loss: 2.314758539199829, Accuracy: 11.734067916870117, Val Loss: 2.2856502532958984, Val Accuracy: 13.40000057220459
Iteration 100, Epoch 1, Loss: 2.302516460418701, Accuracy: 12.360767364501953, Val Loss: 2.2672672271728516, Val Accuracy: 14.800000190734863
Iteration 150, Epoch 1, Loss: 2.2911875247955322, Accuracy: 13.110513687133789, Val Loss: 2.2497103214263916, Val Accuracy: 16.399999618530273
Iteration 200, Epoch 1, Loss: 2.2811481952667236, Accuracy: 14.140235900878906, Val Loss: 2.233107566833496, Val Accuracy: 18.700000762939453
Iteration 250, Epoch 1, Loss: 2.2726693153381348, Accuracy: 14.772161483764648, Val Loss: 2.218217611312866, Val Accuracy: 20.5
Iteration 300, Epoch 1, Loss: 2.2642464637756348, Accuracy: 15.614618301391602, Val Loss: 2.2030749320983887, Val Accuracy: 22.30000114440918
Iteration 350, Epoch 1, Loss: 2.2

In [33]:
model = model_init_fn()
model.compile(optimizer='sgd',
              loss='sparse_categorical_crossentropy',
              metrics=[tf.keras.metrics.sparse_categorical_accuracy])
model.fit(X_train, y_train, batch_size=64, epochs=1, validation_data=(X_val, y_val))
model.evaluate(X_test, y_test)

2024-05-04 22:55:10.093034: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 602112000 exceeds 10% of free system memory.


766/766 ━━━━━━━━━━━━━━━━━━━━ 100s 130ms/step - loss: 1.8597 - sparse_categorical_accuracy: 0.3334 - val_loss: 1.4548 - val_sparse_categorical_accuracy: 0.4950
313/313 ━━━━━━━━━━━━━━━━━━━━ 7s 21ms/step - loss: 1.4928 - sparse_categorical_accuracy: 0.4709


[1.4937865734100342, 0.4674000144004822]

# Использование Keras Functional API

Для реализации более сложных архитектур сети с несколькими входами/выходами, повторным использованием слоев, "остаточными" связями (residual connections) необходимо явно указать входные и выходные тензоры. 

Ниже представлен пример для полносвязной сети. 

In [34]:
def two_layer_fc_functional(input_shape, hidden_size, num_classes):  
    initializer = tf.initializers.VarianceScaling(scale=2.0)
    inputs = tf.keras.Input(shape=input_shape)
    flattened_inputs = tf.keras.layers.Flatten()(inputs)
    fc1_output = tf.keras.layers.Dense(hidden_size, activation='relu',
                                 kernel_initializer=initializer)(flattened_inputs)
    scores = tf.keras.layers.Dense(num_classes, activation='softmax',
                             kernel_initializer=initializer)(fc1_output)

    # Instantiate the model given inputs and outputs.
    model = tf.keras.Model(inputs=inputs, outputs=scores)
    return model

def test_two_layer_fc_functional():
    """ A small unit test to exercise the TwoLayerFC model above. """
    input_size, hidden_size, num_classes = 50, 42, 10
    input_shape = (50,)
    
    x = tf.zeros((64, input_size))
    model = two_layer_fc_functional(input_shape, hidden_size, num_classes)
    
    with tf.device(device):
        scores = model(x)
        print(scores.shape)
        
test_two_layer_fc_functional()

(64, 10)


In [35]:
input_shape = (32, 32, 3)
hidden_size, num_classes = 4000, 10
learning_rate = 1e-2

def model_init_fn():
    return two_layer_fc_functional(input_shape, hidden_size, num_classes)

def optimizer_init_fn():
    return tf.keras.optimizers.SGD(learning_rate=learning_rate)

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 3.1881117820739746, Accuracy: 9.375, Val Loss: 2.848323106765747, Val Accuracy: 12.5
Iteration 50, Epoch 1, Loss: 2.3916869163513184, Accuracy: 25.245098114013672, Val Loss: 2.18837571144104, Val Accuracy: 32.60000228881836
Iteration 100, Epoch 1, Loss: 2.2506513595581055, Accuracy: 28.341585159301758, Val Loss: 1.912186622619629, Val Accuracy: 37.099998474121094
Iteration 150, Epoch 1, Loss: 2.144514799118042, Accuracy: 30.608442306518555, Val Loss: 1.9049874544143677, Val Accuracy: 37.70000076293945
Iteration 200, Epoch 1, Loss: 2.0835964679718018, Accuracy: 32.299442291259766, Val Loss: 1.83794105052948, Val Accuracy: 39.70000076293945
Iteration 250, Epoch 1, Loss: 2.046450614929199, Accuracy: 33.098854064941406, Val Loss: 1.8482515811920166, Val Accuracy: 37.70000076293945
Iteration 300, Epoch 1, Loss: 2.006763458251953, Accuracy: 34.03239440917969, Val Loss: 1.8402645587921143, Val Accuracy: 38.60000228881836
Iteration 350, Epoch 1, Loss: 1.974068760871

Поэкспериментируйте с архитектурой сверточной сети. Для вашего набора данных вам необходимо получить как минимум 70% accuracy на валидационной выборке за 10 эпох обучения. Опишите все эксперименты и сделайте выводы (без выполнения данного пункта работы приниматься не будут). 

Эспериментируйте с архитектурой, гиперпараметрами, функцией потерь, регуляризацией, методом оптимизации.  

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/BatchNormalization#methods https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/Dropout#methods

In [ ]:
class CustomConvNet(tf.keras.Model):
    def __init__(self):
        super(CustomConvNet, self).__init__()
        ############################################################################
        # TODO: Construct a model that performs well on CIFAR-10                   #
        ############################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        
        # Дробавляем входной сверточный слой с функцией активации 'relu' и 16 фильтрами
        self.conv1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(32, 32, 3))
        # Добавим скрытый сверточный слой с ядром 3 x 3 и функцией активации 'relu'
        self.conv2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu')
        # Максимальный пулинг 2 х 2
        self.pool1 = tf.keras.layers.MaxPooling2D((2, 2))
        # Добавляем дропаут в 0.25
        self.dropout1 = tf.keras.layers.Dropout(0.25)
        # Добавим скрытый сверточный слой с ядром 3 x 3 и функцией активации 'relu'
        self.conv3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu')
        # Максимальный пулинг 2 х 2
        self.pool2 = tf.keras.layers.MaxPooling2D((2, 2))
        # Слой для развертывания
        self.flatten = tf.keras.layers.Flatten()
        # Полносвязный слой с функцией активации 'relu'
        self.dense1 = tf.keras.layers.Dense(128, activation='relu')
        # Добавляем дропаут в 0.4
        self.dropout2 = tf.keras.layers.Dropout(0.4)
        # Выходной полносвязный слой с сигмоидой в качестве функции активации
        self.dense2 = tf.keras.layers.Dense(10, activation='softmax')

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ############################################################################
        #                            END OF YOUR CODE                              #
        ############################################################################
    
    def call(self, input_tensor, training=False):
        ############################################################################
        # TODO: Construct a model that performs well on CIFAR-10                   #
        ############################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        x = self.conv1(input_tensor)
        x = self.conv2(x)
        x = self.pool1(x)

        if training:
            x = self.dropout1(x, training=training)

        x = self.conv3(x)
        x = self.pool2(x)
        x = self.flatten(x)
        x = self.dense1(x)

        if training:
            x = self.dropout2(x, training=training)

        x = self.dense2(x)

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ############################################################################
        #                            END OF YOUR CODE                              #
        ############################################################################
        return x


print_every = 500
num_epochs = 4

model = CustomConvNet()

def model_init_fn():
    return CustomConvNet()

def optimizer_init_fn():
    learning_rate = 1e-3
    return tf.keras.optimizers.Adam(learning_rate) 

train_part34(model_init_fn, optimizer_init_fn, num_epochs=num_epochs, is_training=True)

Iteration 0, Epoch 1, Loss: 2.3610124588012695, Accuracy: 14.0625, Val Loss: 2.3511886596679688, Val Accuracy: 12.199999809265137
Iteration 500, Epoch 1, Loss: 1.6444170475006104, Accuracy: 40.25074768066406, Val Loss: 1.285671591758728, Val Accuracy: 54.5
Iteration 1000, Epoch 2, Loss: 1.2148468494415283, Accuracy: 57.06781768798828, Val Loss: 1.0373276472091675, Val Accuracy: 65.9000015258789
Iteration 1500, Epoch 2, Loss: 1.1421825885772705, Accuracy: 59.619476318359375, Val Loss: 1.0032247304916382, Val Accuracy: 65.5
Iteration 2000, Epoch 3, Loss: 1.0070418119430542, Accuracy: 64.65885162353516, Val Loss: 0.9048417210578918, Val Accuracy: 68.19999694824219
Iteration 2500, Epoch 4, Loss: 0.9400681853294373, Accuracy: 66.77955627441406, Val Loss: 0.8691628575325012, Val Accuracy: 69.5
Iteration 3000, Epoch 4, Loss: 0.9139112830162048, Accuracy: 67.76093292236328, Val Loss: 0.8069267272949219, Val Accuracy: 73.0


Опишите все эксперименты, результаты. Сделайте выводы.

In [ ]:
Архитектура сети состоит из сверточных слоев, максимального пулинга, droput, flatten и полносвязных слоев.
На одинаковых настройках были протестированы различные оптимизаторы: SGD, Adam, RMSprop.
Первый показал наихудшие результаты - за 10 эпох точность была в районе 30%.
Последние два показали примерно одинаковые результаты, но в большинстве своем Adam был точнее.
На входном и скрытом слоях сравнивались функции активации relu и sigmoid, а на входном слое тестировались sigmoid и softmax.
При комбинации sigmoid + softmax + Adam получился наихудший результат - для получения необходимой точности потребовалось в районе 13 эпох.
При комбинации relu + softmax + Adam был получен наилучший результат - 4 эпохи.